#Bazy danych oraz źródła

Baza danych związana z Action Units oraz mruganiem jest częścią osobnego projektu związanego z doktoratem, obecnie jeszcze nie opublikowanego. Na jej powstanie składają się trzy osobne algorytmy do przetwarzania wideo, z wykorzystaniem bibliotek mediapipe, keras_facenet, MTCNN, deepface. Dołączone zostały do niej wyniki analiz wskaźników ekonomicznych - w przypadku tego projektu zostały użyte NEER (Nominal Effective Exchange Rate) oraz FX (Foreign Exchange).

Baza z komunikacją niewerbalną oraz wybranymi wskaźnikami ekonomicznymi zawiera 108 kolumn z różnymi wariancjami AU, 4 kolumny z mruganiem oraz 22 kolumny dotyczące danych rynkowych, walutowych i identyfikacyjnych. Baza danych ma 136 wierszy, stanowiąc liczbę równą ilości przeanalizowanych konferencji. Dla możliwości przeprowadzenia badania została przygotowana próbka zawierająca 50 wierszy. Próbka ta stanowi reprezentatywny wycinek bazy panelowej zachowujący strukturę cech i relacje czasowe.

Łączenie bazy wraz z kursami walutowymi oraz generowanie próbki

In [4]:
import pandas as pd
import numpy as np
from scipy import stats
import os

#Plik zawierający AU, mruganie, NEER i FX
final_df_monthly = pd.read_csv('au_eye_macro_financial_complete_monthly_v3.csv')

if 'month_str' not in final_df_monthly.columns and 'month' in final_df_monthly.columns:
    final_df_monthly['month_str'] = final_df_monthly['month'].astype(str)

def analyze_single_currency(fx_file_path, facial_metrics_df, currency_code="CHF"):
    try:
        fx_data = pd.read_csv(fx_file_path)
        date_col = [c for c in fx_data.columns if 'date' in c.lower() or 'time' in c.lower()][0]
        rate_col = [c for c in fx_data.columns if 'rate' in c.lower() or 'value' in c.lower() or currency_code.lower() in c.lower()][0]

        fx_data = fx_data[[date_col, rate_col]].copy()
        fx_data.columns = ['date', 'rate']
        fx_data['date'] = pd.to_datetime(fx_data['date'])
        fx_data['month_str'] = fx_data['date'].dt.to_period('M').astype(str)

        #Obliczanie stóp zwrotu dla zmienności
        fx_data['log_return'] = np.log(fx_data['rate'] / fx_data['rate'].shift(1))

        fx_monthly = fx_data.groupby('month_str').agg({
            'rate': 'mean',
            'log_return': 'std'
        }).rename(columns={
            'rate': f'EUR{currency_code}_Level',
            'log_return': f'EUR{currency_code}_Vol'
        }).reset_index()

        #Łączenie z bazą (AU, Blink, NEER, FX_macro)
        merged = pd.merge(facial_metrics_df, fx_monthly, on='month_str', how='inner')
        merged['Currency_Tag'] = currency_code

        return merged
    except Exception as e:
        print(f"Błąd przy {currency_code}: {e}")
        return pd.DataFrame()
currencies = ['CHF', 'USD', 'GBP', 'JPY', 'CNY']
all_results_list = []

for curr in currencies:
    file_path = f'ecb_fx_{curr.lower()}_2010.csv'
    if os.path.exists(file_path):
        res = analyze_single_currency(file_path, final_df_monthly, curr)
        if not res.empty:
            all_results_list.append(res)

#GENEROWANIE PRÓBKI (50 WIERSZY)
if all_results_list:
    combined_df = pd.concat(all_results_list, ignore_index=True)

    base_cols = ['month_str', 'Currency_Tag']

    au_cols = [c for c in combined_df.columns if 'AU' in c]

    blink_cols = [c for c in combined_df.columns if 'blink' in c.lower()]

    macro_cols = [c for c in combined_df.columns if 'neer' in c.lower() or 'fx_' in c.lower()]

    specific_fx_cols = [c for c in combined_df.columns if '_Level' in c or '_Vol' in c]

    final_columns = sorted(list(set(base_cols + au_cols + blink_cols + macro_cols + specific_fx_cols)))

    sample_50 = combined_df[final_columns].tail(50)

    sample_50.to_csv('final_fx_facial_sample.csv', index=False)

    print(f"Liczba kolumn: {len(final_columns)}")
    print(f"Kolumny AU: {len(au_cols)}")
else:
    print("Błąd: Nie znaleziono danych do połączenia.")

Liczba kolumn: 134
Kolumny AU: 108


In [5]:
if all_results_list:
  combined_df = pd.concat(all_results_list, ignore_index=True)
  base_cols = ['month_str', 'Currency_Tag']
  au_cols = [c for c in combined_df.columns if 'AU' in c]
  blink_cols = [c for c in combined_df.columns if 'blink' in c.lower()]
  macro_cols = [c for c in combined_df.columns if 'neer' in c.lower() or 'fx_' in c.lower()]
  specific_fx_cols = [c for c in combined_df.columns if '_Level' in c or '_Vol' in c]
  final_columns = sorted(list(set(base_cols + au_cols + blink_cols + macro_cols + specific_fx_cols)))
  data = combined_df[final_columns]
  data.to_csv('final_data.csv', index=False)
  print(f"Liczba kolumn: {len(final_columns)}")
  print(f"Kolumny AU: {len(au_cols)}")
else:
  print("Błąd: Nie znaleziono danych do połączenia.")

sample_data = pd.read_csv('final_data.csv') #plik całościowy do przeprowadzonej przeze mnie analizy

Liczba kolumn: 134
Kolumny AU: 108


##Wykaz bibliotek i narzędzi użytych w projekcie

###Przetwarzanie i Analiza Danych
* Pandas - Podstawa manipulacji danymi, łączenia baz oraz operacji na szeregach czasowych.
* NumPy - Obliczenia macierzowe, operacje logarytmiczne i statystyczne.
* os - Zarządzanie ścieżkami plików, automatyzacja wczytywania baz danych i tworzenie folderów dla modeli produkcyjnych.
* warnings: Kontrola komunikatów systemowych.
* StandardScaler / MinMaxScaler (Scikit-Learn) -  Normalizacja danych behawioralnych i rynkowych (kluczowa dla stabilności sieci neuronowych i SVR).

###Statystyka i Weryfikacja Hipotez
* SciPy (stats) - Obliczanie korelacji Pearsona.
* Statsmodels (grangercausalitytests) - Kluczowe narzędzie do przeprowadzenia testów przyczynowości Grangera, potwierdzających rolę mimiki jako wskaźnika wyprzedzającego.

###Machine Learning (Modelowanie i Tuning)
* Scikit-Learn - Modele:
  1. LinearRegression,
  2. Ridge,
  3. SVR,
  4. RandomForestRegressor,
  5. GradientBoostingRegressor,
  6. MLPRegressor (Neural Network).
* Klasyfikatory:
  1. LogisticRegression,
  2. RidgeClassifier,
  3. SVC,
  4. RandomForestClassifier,
  5. GradientBoostingClassifier,
  6. MLPClassifier.
* Tuning:
  1. RandomizedSearchCV,
  2. GridSearchCV.
* Selekcja cech:
  1. LassoCV,
  2. SelectFromModel,
  3. PCA (Analiza Składowych Głównych).
* Walidacja:
  1. TimeSeriesSplit,
  2. r2_score,
  3. accuracy_score,
  4. confusion_matrix.
###Wizualizacja i Operacjonalizacja:
* Matplotlib / Seaborn: Generowanie macierzy, wykresów oraz analizy reszt (residuals).
* Joblib: Narzędzie do zapisu wytrenowanych modeli, scalerów i obiektów PCA do plików .pkl w celu ich późniejszego wykorzystania.

##Lista wykorzystywanych baz danych:
* CHF (Frank szwajcarski) https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/eurofxref-graph-chf.en.html
* CNY (Juan chiński) https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/eurofxref-graph-cny.en.html
* GBP (Funt brytyjski) https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/eurofxref-graph-gbp.en.html
* JPY (Jen japoński) https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/eurofxref-graph-jpy.en.html
* NEER (Nominalne efektywne kursy wymiany) https://www.ecb.europa.eu/stats/balance_of_payments_and_external/eer/html/index.en.html
* USD (Dolar amerykański) https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/eurofxref-graph-usd.en.html
* YouTube – Playlista ECB Press Conferences https://www.youtube.com/playlist?list=PL5C2C2383444CDA3D
